In [2]:
import gym

In [5]:
env = gym.make('MountainCar-v0')
path_to_recording = './tmp/MountainCar-v0-dqn'

[2017-02-12 12:30:27,447] Making new env: MountainCar-v0


In [1]:
def DQN(input_nodes, output_nodes, hidden_nodes = 10):
    """
    4 layer(2 hidden layer) network
    input: state array
    output: estimated q_values(future reward for choosing that action)
    """

    state = tf.placeholder(dtype = tf.float32, shape = (None, input_nodes))
    Q_actuals = tf.placeholder(dtype=tf.float32, shape = (None, output_nodes))

    w1 = tf.Variable(tf.truncated_normal([input_nodes, hidden_nodes], stddev = 0.1))
    b1 = tf.Variable(tf.truncated_normal([hidden_nodes], stddev = 0.1))
    h1 = tf.nn.relu(tf.matmul(state, w1) + b1)

    w2 = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=0.1))
    b2 = tf.Variable(tf.truncated_normal([hidden_nodes], stddev=0.1))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)

    w3 = tf.Variable(tf.random_normal([hidden_nodes, output_nodes]))
    b3 = tf.Variable(tf.zeros([output_nodes]))
    Q_est = tf.matmul(h2, w3) + b3

    loss = tf.nn.l2_loss(Q_est - Q_actuals)
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    return optimizer, state, Q_actuals, Q_est, loss, w1, w2, w3

dqn_opt, dqn_state, dqn_q_actuals, dqn_q_est, dqn_loss, w1, w2, w3 = DQN(input_nodes=2, output_nodes=3)


class ExperienceReplay:
    
    def __init__(self, gamma = 0.97,  batch_size=128, max_size=50000):
        self.memory = []
        self.max_size = max_size
        self.batch_size = batch_size
        self.gamma = gamma
        
    def add_observation(self, step, obs, action, reward, is_done, new_obs, future_reward):
        if len(self.memory) > self.max_size:
            self.memory.pop(0) #remove older samples    
        self.memory.append((step, obs, action, reward, is_done, new_obs, future_reward))
    
    def get_mini_batch(self):
        samples = np.random.choice(len(self.memory), self.batch_size)
        return [self.memory[idx] for idx in samples]
    
    def train(self,sess):
        batch = self.get_mini_batch()
        step_batch, obs_batch, action_batch, reward_batch, is_done_batch, \
        new_obs_batch, future_reward_batch = map(np.array, zip(*batch))
        
        #get next q values by running DQN on Next States
        #calculate max next q
        #update current state with (reward  + max next q - old q)  
        
        q_vals = sess.run(dqn_q_est, feed_dict = {dqn_state : obs_batch})        
        new_obs_q_vals = sess.run(dqn_q_est, feed_dict = {dqn_state : new_obs_batch})
        maxQ = np.max(new_obs_q_vals, axis = 1)
        
        target_q_vals = q_vals[:]
        target_q_vals[np.arange(self.batch_size), action_batch] = future_reward_batch \
                                                                + ((1.0 - is_done_batch.astype(int)))*self.gamma*maxQ
        
        l, _ = sess.run([dqn_loss, dqn_opt], feed_dict={dqn_state : obs_batch, 
                                                        dqn_q_actuals: target_q_vals
                                                       })
        return l


TypeError: DQN() takes at least 2 arguments (0 given)

In [8]:
num_episodes = 10

for episode in range(num_episodes):
    
    while True:
        next_obs, reward, is_done, _ = env.step(1)
        env.render()

RuntimeError: Canvas has not been attached